<a href="https://colab.research.google.com/github/lazlopeabody/ml_study/blob/master/transfer_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

USE PRE-Trained Models

In [1]:
import tensorflow

In [2]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers, Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import urllib
import os
import zipfile

In [9]:
class myCallback(tensorflow.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs=None):
        if(logs.get('accuracy')>0.98):
            print("\nTraining Accuracy == 98%")
            self.model.stop_training = True

callbacks = myCallback()

In [5]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"

In [6]:
urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x7f4182460150>)

In [12]:
training_file_name = '/content/sample_data/dogs-vs-cats/train.zip'
training_dir = '/content/sample_data/dogs-vs-cats/_train/'
val_file_name = '/content/sample_data/dogs-vs-cats/test1.zip'
val_dir = '/content/sample_data/dogs-vs-cats/_test1' 

In [11]:
val_zip_ref = zipfile.ZipFile(val_file_name, 'r')
val_zip_ref.extractall(val_dir)
val_zip_ref.close()

In [13]:
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

Below is wehere we will tweak parameters to get after accuracy in the model, training the model to be more accuracte. 

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',

)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    batch_size=100,
    class_mode='binary',
    target_size=(150,150)
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    batch_size=100,
    class_mode='binary',
    target_size=(150,150)
)

Found 25000 images belonging to 1 classes.
Found 12500 images belonging to 1 classes.


Transfer Learning, using pre-trained google model

In [17]:
pre_trained_model = InceptionV3(
    input_shape=(150,150,3),
    include_top=False,
    weights=None
)

In [18]:
pre_trained_model.load_weights(weights_file)

In [19]:
for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [20]:
# flatten output layer to 1 dimension
x = layers.Flatten()(last_output)
# add full connected layer w/ 1024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# add final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

In [21]:
model = Model(pre_trained_model.input, x)
model.compile(
    optimizer=RMSprop(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    verbose=1,
    callbacks=[myCallback()]
)

Epoch 1/30
 42/250 [====>.........................] - ETA: 16:17 - loss: 0.0210 - accuracy: 0.9826